This notbooks does a GGOR run using the python code in gg_mflow.py. The code in this notebook is essentially the same as in the mentioned gg_mflow.py after "if __name__ == __main__', but commented for explanation.

Theo Olsthoorn, Aug. 2023

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import flopy
import gg_mflow as gg
from importlib import reload

In [2]:
reload(gg)

<module 'gg_mflow' from '/Users/Theo/GRWMODELS/python/GGOR/src/numeric/gg_mflow.py'>

Set parameter `test` to `False` to use the database, else generated test meteo  data and a selection of parcels from the database are used.

In [3]:
test=False
case = 'AAN_GZK'

Setup the data and the model

In [4]:
# Parameters to generate the model. Well use this as **kwargs
GGOR_home = os.path.expanduser('~/GRWMODELS/python/GGOR') # home directory
    

#GGOR directory structure
dirs = gg.Dir_struct(GGOR_home, case=case)

#Get the meteo data from an existing file or directly from KNMI
meteo_data = gg.knmi.get_weather(stn=240, start='20100101', end='20191231',
                                  folder=dirs.meteo)

# Add columns "summer' and "hyyear" to it"
tdata = gg.handle_meteo_data(meteo_data, summer_start=4, summer_end=10)

if test:
    tdata = gg.gen_testdata(tdata=tdata,
                          RH  =(270, 0.0, 0.002, 0.004),
                          EV24=(180, 0.0, 0.001, 0.002),
                          )

    parcel_data = gg.get_test_parcels(os.path.join(
                            dirs.case, 'pdata_test.xlsx'), 'parcel_tests1')

    # Special test
    parcel_data = parcel_data.iloc[0:4]
else:
    # Bofek data, coverting from Bofek codes to the soil properties kh, kv and sy.
    # The BOFEK column represents a dutch standardized soil type. It is used.
    # The corresponding values for 'kh', 'kv' and 'Sy' are currently read from
    # an Excel worksheet into a pandas DataFrame (becoming a table)
    bofek = pd.read_excel(os.path.join(dirs.bofek, "BOFEK eenheden.xlsx"),
                          sheet_name = 'bofek', index_col=0, engine="openpyxl")
    # Create a GGOR_modflow object and get the upgraded parcel_data from it,
    # excluding parcels that are too small or too wide
    parcel_data = gg.GGOR_data(defaults=defaults, bofek=bofek, BMINMAX=(5, 250),
            GGOR_home=GGOR_home, case=case).data

File </Users/Theo/GRWMODELS/python/GGOR/data/meteo/weer240_20100101_20191231.txt> exists, download was skipped.


ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.

Run MODFLOW. The input arrays are in the returned dicts.

In [ ]:
par, spd, bdd, gr =  gg.run_modflow(dirs=dirs, parcel_data=parcel_data, tdata=tdata,
                                 use_w_not_c=True)

Get the modflow-computed heads and cell by cell flows

In [ ]:

heads = gg.Heads_obj(dirs, tdata=tdata, IBOUND=par['IBOUND'], gr=gr)

watbal = gg.Watbal_obj(dirs,
                       tdata=tdata,
                       parcel_data=parcel_data,
                       IBOUND=par['IBOUND'],
                       gr=gr)

Open HDS file and plot heads (with GXG)

In [ ]:
 if test:
    for tst in set(parcel_data['Test'].values):
        selection = list(parcel_data.index[parcel_data['Test'] == tst])
        test_vals_str = '{}'.format(', '.join(
            [str(tv) for tv in parcel_data[tst].iloc[selection]]))

        titles=['Parcel averaged heads, testvar {} in [{}]'.format(tst, test_vals_str),
                'Parcel averaged heads, testvar {} in [{}]'.format(tst, test_vals_str)]
        ax = heads.plot(tdata=tdata,
                       parcel_data=parcel_data,
                       selection=selection,
                       titles=titles,
                       size_inches=(14, 8))
        if False:
            ax = watbal.plot(parcel_data=parcel_data,
                             tdata=tdata,
                             selection=selection[0],
                             sharey=True)
else:
    selection = [0, 1, 2, 3]
    titles=['Parcel averaged heads', 'Parcel averaged heads']
    ax = heads.plot(tdata=tdata,
           parcel_data=parcel_data,
           selection=selection,
           titles=titles,
           size_inches=(14, 8))
    ax = watbal.plot(parcel_data=parcel_data,
                     tdata=tdata,
                     selection=None,   # over all parcels
                     sharey=True)

Compute and show the running water buget over all parcells

In [ ]:
ax = gg.watbal.plot(parcel_data=parcel_data,
                     tdata=tdata,
                     selection=None,   # over all parcels
                     sharey=True)
